# Exercise 6

# Setup to run the notebook

- Ensure that `python3` is available in the system with python version 3.10.
- Create a virtual env - `python3 -m venv env`
- Activate the env - `source env/bin/activate`
- Run the code cells for observing results.

## 1: The ElGamal cryptosystem

In [3]:
import random

In [4]:
# Helper function
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Helper function
def find_primitive_root(q):
    for g in range(2, q):
        if gcd(g, q) == 1:
            return g
    return None

# Key generation for the ElGamal cryptosystem
def e_keygen(q):
    g = find_primitive_root(q)
    if g is None:
        return None, None
    x = random.randint(1, q-2)
    h = pow(g, x, q)

    public_key = (q, g, h)
    private_key = x

    return public_key, private_key

In [5]:
# Key encryption for the ElGamal cryptosystem
def e_encrypt(msg, pk):
    q, g, h = pk
    k = random.randint(1, q-2)

    c1 = pow(g, k, q)
    c2 = (msg * pow(h, k, q)) % q

    return c1, c2

In [6]:
# Helper function
def modinv(a, m):
    # This function returns the inverse of a modulo m
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

# Helper function
def egcd(a, b):
    # Extended Euclidean Algorithm
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# Key decryption for the ElGamal cryptosystem
def e_decrypt(c, x, pk):
    c1, c2 = c
    q, g, h = pk

    s = pow(c1, x, q)
    s_inv = modinv(s, q)
    msg = (c2 * s_inv) % q
    return msg

In [7]:
# Test the ElGamal cryptosystem for generating keys, encrypting and decrypting
q = 2**127 - 1

sk, pk = e_keygen(q)
c = e_encrypt(50, sk)

e_decrypt(c, pk, sk)

50

In [8]:
# Multiplication of ElGamal ciphertexts
def e_mult(c1, c2, pk):
    q, g, h = pk
    c11, c12 = c1
    c21, c22 = c2

    # Multiply the first parts of the ciphertexts
    new_c1 = (c11 * c21) % q

    # Multiply the second parts of the ciphertexts
    new_c2 = (c12 * c22) % q

    return new_c1, new_c2

In [9]:
# Test cases for showing correctness of the implementation

# Test 1
q = 2**127 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(5, pk)
c2 = e_encrypt(10, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 50

# Test 2
q = 2**227 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(5**25, pk)
c2 = e_encrypt(10**25, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 2980232238769531250000000000000000000000000


# Test 3
q = 2**327 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(28**50, pk)
c2 = e_encrypt(329**50, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 227649946206217048085015677443757528434845177408940388249825297116688924748709775450598383345711419

## Question 2: Application of partially homomorphic encryption: Paillier (5 points)
Below you are given an implementation of the partially homomorphic Pailler encryption scheme. 

Which operations does the Pailler encryption scheme support? Which data types does it operate on?

Use the Paillier implementation to write functions that allow a server to compute the (encrypted) total price of a shopping cart based on encrypted prices and cleartext quantities.

Complete the following functions:
- `shopping_cart_client`, which implements a client that sets up homomorphic encryption(generates keys), encrypts the prices of shopping cart items, lets the server compute the total cost of the shopping cart, decrypts and returns the total cost
- `encrypted_price_calculator`, which implements a server that computes the encrypted total cost of the shopping cart and returns the value to the client

1. It supports the following operations
- Homomorphic Addition of Ciphertexts
- Homomorphic Multiplication of an Encrypted Number by a Plaintext Number

2. The Paillier cryptosystem operates on integers.

In [ ]:
%pip install -U pip wheel setuptools
%pip install numpy
%pip install phe


In [8]:
PrivateKey = namedtuple("PrivateKey", ["lam", "mu"])
PublicKey = namedtuple("PublicKey", ["g", "n", "n_squared"])

DEFAULT_BIT_LENGTH = 32


def generate_primes(n: int) -> List[int]:
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Input n>=6, Returns an array of primes, 2 <= p < n """
    sieve = np.ones(n // 3 + (n % 6 == 2), dtype=bool)
    for i in range(1, int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[k * k // 3 :: 2 * k] = False
            sieve[k * (k - 2 * (i & 1) + 4) // 3 :: 2 * k] = False
    primes = np.r_[2, 3, ((3 * np.nonzero(sieve)[0][1:] + 1) | 1)]
    return [int(n) for n in primes]


def L(n: int, x: int) -> int:
    return (x - 1) // n


def create_key_pair(
    bit_length: int = DEFAULT_BIT_LENGTH,
) -> Tuple[PrivateKey, PublicKey]:
    primes = generate_primes(2 ** (bit_length // 2))

    p = secrets.choice(primes)
    q = secrets.choice(primes)
    n = p * q

    while p == q or n.bit_length() != bit_length or np.gcd(n, (p - 1) * (q - 1)) != 1:
        p = secrets.choice(primes)
        q = secrets.choice(primes)
        n = p * q

    n_squared = n ** 2
    g = secrets.randbelow(n_squared - 1) + 1
    public_key = PublicKey(g, n, n_squared)

    lam = int(np.lcm(p - 1, q - 1))

    try:
        mu = pow(L(n, pow(g, lam, n_squared)), -1, n)
    except ValueError:
        return create_key_pair(bit_length)

    private_key = PrivateKey(lam, mu)
    return private_key, public_key


def encrypt(public_key: PublicKey, plaintext: int) -> int:
    g, n, n_squared = public_key
    r = secrets.randbelow(n)
    return (pow(g, plaintext, n_squared) * pow(r, n, n_squared)) % n_squared


def decrypt(private_key: PrivateKey, public_key: PublicKey, ciphertext: int) -> int:
    lam, mu = private_key
    _, n, n_squared = public_key
    return (L(n, pow(ciphertext, lam, n_squared)) * mu) % n


def add(public_key: PublicKey, ciphertext_a: int, ciphertext_b: int) -> int:
    return (ciphertext_a * ciphertext_b) % public_key.n_squared


def multiply(public_key: PublicKey, ciphertext_a: int, plaintext_b: int) -> int:
    if plaintext_b == 0:
        return encrypt(public_key, 0)

    if plaintext_b == 1:
        encrypted_zero = encrypt(public_key, 0)
        return add(public_key, ciphertext_a, encrypted_zero)

    return pow(ciphertext_a, plaintext_b, public_key.n_squared)

In [9]:
def encrypted_price_calculator(
    public_key: PublicKey,
    encrypted_cart: List[Tuple[int, int]],
) -> int:
    total_encrypted = encrypt(public_key, 0)  # Start with an encrypted value of 0

    for encrypted_price, quantity in encrypted_cart:
        # Multiply each encrypted price by its quantity and add to the total
        total_encrypted = add(public_key, total_encrypted, multiply(public_key, encrypted_price, quantity))

    return total_encrypted

In [10]:
def shopping_cart_client(cart):
    # Generate keys
    private_key, public_key = create_key_pair()

    # Encrypt the prices in the cart
    encrypted_cart = [(encrypt(public_key, price), quantity) for price, quantity in cart]

    # Let the server compute the encrypted total cost
    encrypted_total = encrypted_price_calculator(public_key, encrypted_cart)

    # Decrypt the total cost
    total_cost = decrypt(private_key, public_key, encrypted_total)

    return total_cost

In [11]:
# TEST CASE
cart = [
    # (price, quantity)
    (2000, 1),
    (120, 5),
    (1999, 3),
]

total_cost = shopping_cart_client(cart)
expected_price = 8597
assert total_cost == expected_price

## Question 3: Application of partially homomorphic encryption: ElGamal (5 points)
Change the shopping cart scenario to work with the ElGamal cryptosystem.

Which changes did you have to make? What does this mean for the client (which advantages and/or disadvantages does the client have)?

*Hint:* Which operations can the server perform? Which operations would be left for the client? Which operands can be encrypted, in comparison to the Paillier implementation?

In [12]:
def e_multiply(pk, encrypted_value, scalar):
    q, g, h = pk
    return pow(encrypted_value, scalar, q)

def e_encrypted_price_calculator(price, quantity, pk):
    encrypted_total = e_multiply(pk, price, quantity)
    return encrypted_total

In [13]:
def e_shopping_cart_client(cart):
    # Generate the public and private keys for ElGamal encryption
    public_key, private_key = e_keygen(q)
    # Encrypt the prices and calculate encrypted total for each item
    total_cost = 0
    for price, quantity in cart:
        encrypted_price = e_encrypt(price, public_key)
        # Decrypt immediately (since ElGamal doesn't support homomorphic addition)
        decrypted_price = e_decrypt(encrypted_price, private_key, public_key)
        total_cost += decrypted_price * quantity
    return total_cost

In [14]:
# TEST CASE
cart = [
    # (price, quantity)
    (2000, 1),
    (120, 5),
    (1999, 3),
]


total_cost = e_shopping_cart_client(cart)
expected_price = 8597
assert total_cost == expected_price

1. In EIGamal cryptosystem, we can perform the multiplication of two encrypted values or we can multiply an encrypted value by a plaintext value. We cannot directly add two encrypted values, which is a key operation needed for calculating the total price in the shopping cart.
2. The server can only multiply encrypted values, not add them. This means the server can't directly calculate the total price if both price and quantity are encrypted. 
3. The client could encrypt a total price for each item (price multiplied by quantity) before sending it to the server.
4. The client eventually has to decrypt the final aggregated result, requiring two steps of computation (before sending to the server and after receiving the result).

## Question 4: Fully homomorphic encryption (5 points)
1. Use the `Concrete Numpy` or `Pyfhel` library to compute the mean over a list of 6 encrypted integers.
1. Implement a second version that gives you a mean with a precision of 2 decimal digits.
1. How can you obtain the mean of a list with 7 encrypted integers?

Clearly document the limitations of your implementations.

In [15]:
from Pyfhel import Pyfhel


def homomorphic_mean_ckks(HE, x):
    # Start with an encrypted zero
    encrypted_sum = HE.encrypt(0)
    for value in x:
        temp_ctxt = HE.encrypt(value)
        encrypted_sum += temp_ctxt

    # Decrypt the sum
    decrypted_sum = HE.decrypt(encrypted_sum, decode=False)
    integer_sum = HE.decode(decrypted_sum)

    # Calculate the mean
    mean = integer_sum / len(x)
    return mean[:3]

    
def generate_ckks_context():
    HE = Pyfhel()
    try:
        HE.contextGen(scheme='BFV', n=4096, t_bits=20, sec=128)
        HE.keyGen()
    except Exception as e:
        print(f"Error initializing Pyfhel: {e}")
        raise

    return HE


ModuleNotFoundError: No module named 'Pyfhel'

In [ ]:
example = [1, 7, 4, 5, 5, 4]
HE = generate_ckks_context()
homomorphic_evaluation = homomorphic_mean_ckks(HE, example)
clear_evaluation = np.mean(example)
print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {homomorphic_evaluation}")

In [ ]:
def homomorphic_mean_2(HE, x):
    # Start with an encrypted zero
    encrypted_sum = HE.encrypt(0)
    for value in x:
        temp_ctxt = HE.encrypt(value)
        encrypted_sum += temp_ctxt

    # Decrypt the sum
    decrypted_sum = HE.decrypt(encrypted_sum, decode=False)
    integer_sum = HE.decode(decrypted_sum)

    # Calculate the mean
    mean = np.around(integer_sum / len(x), 2)
    return mean[:3]
    
def generate_ckks_context():
    HE = Pyfhel()
    try:
        HE.contextGen(scheme='BFV', n=4096, t_bits=20, sec=128)
        HE.keyGen()
    except Exception as e:
        print(f"Error initializing Pyfhel: {e}")
        raise

    return HE

In [ ]:
example = [1, 7, 4, 5, 5, 4]
HE2 = generate_ckks_context()
homomorphic_evaluation = homomorphic_mean_2(HE2, example)
clear_evaluation = np.mean(example)
print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {homomorphic_evaluation}")

1. To compute the mean of a list of 7 encrypted integers, We would modify the mean to divide by 7 instead of 6. The same principle applies for precision adjustment.

2. Limitations:

- Precision: The implementation provides only integer results due to the limitations of homomorphic encryption in handling floating-point operations.

- Performance and Complexity: Homomorphic encryption is computationally expensive and not suitable for very large datasets or highly complex operations.

- Encrypted Data Size: The size of the encrypted data is significantly larger than the plaintext, which may lead to increased storage and transmission costs.